In [1]:
from text2vec import SentenceModel
from flask import Flask
from flask import render_template
from flask import request
from qdrant_client import QdrantClient

/Users/tianlimin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
#os.chdir('/root/ChatBot/QA')
print(os.getcwd())

/Users/tianlimin/vector based model/server


In [3]:
# 加载向量化模型
t2v_model = SentenceModel("/Users/tianlimin/vector based model/mode/text2vec_cmed")

2023-12-29 10:33:48.595 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cpu


In [4]:
# 导入数据库key
with open('key.txt','r') as f:
    key = f.read()

In [5]:
# 文本向量化
def to_embeddings(text):
    sentence_embeddings = t2v_model.encode(text)
    return sentence_embeddings

In [6]:
# load chatglm model
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("/Users/tianlimin/vector based model/mode/chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("/Users/tianlimin/vector based model/mode/chatglm-6b", trust_remote_code=True).half().to('mps')
model = model.eval()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.11s/it]


In [7]:
def ask_glm(text,hsitory=[]):
    response, history = model.chat(tokenizer, text, history=hsitory)
    return response,history

In [9]:
print(ask_glm('What about the headache?')[0])

Headaches are a common condition that can affect anyone, and they can be caused by a variety of factors. Some common causes of headaches include:

1. Migraines: These are severe, persistent headaches that can be accompanied by other symptoms such as nausea, vomiting, and sensitivity to light and sound.
2. Tension headaches: These are often the most common type of headache, and they are caused by muscle tension and stress.
3. Sinus headaches: These are headaches that are caused by inflammation or congestion in the nasal passages.
4.  Cluster headaches: These are severe, intense headaches that are often accompanied by feelings of intense fear or panic.
5. Migraine-like headaches: These headaches may have similar symptoms to migraines, but they are not actually migraines.

It is important to note that headaches can be caused by a variety of factors, and they may not always be related to any one underlying medical condition. If you are experiencing frequent or severe headaches, it is impor

In [8]:
def prompt(question, answers):
    q = '请分析用户故事,根据提示的答案写出用户故事的分数和评价，测试\n' 
    for index, answer in enumerate(answers):
        q += str(index + 1) + '. ' + str(answer['text']) + '\n'
    q = q+"问题：%s || 答案：" % question

    return q


In [9]:
def query(text):
    client = QdrantClient(
        url="https://9759a1d6-6cf2-4049-bd61-a5cb09ee44f3.us-east4-0.gcp.cloud.qdrant.io:6333", 
        api_key=key,
    )
    collection_name = "questions"
    
    vector = to_embeddings(text)
    """
    取搜索结果的前三个，如果想要更多的搜索结果，可以把limit设置为更大的值
    """
    search_result = client.search(
        collection_name=collection_name,
        query_vector=vector.tolist(),
        limit=1,
        search_params={"exact": False, "hnsw_ef": 128}
    )
    answers = []
 
    """
    每个匹配的相关摘要只取了前300个字符，如果想要更多的相关摘要，可以把这里的300改为更大的值
    """
    for result in search_result:
        if len(result.payload["text"]) > 1000:
            summary = result.payload["text"][:1000]
        else:
            summary = result.payload["text"]
        answers.append({ "text": summary})
    promptMessage=prompt(text, answers)
    print(promptMessage)
    response,history =  ask_glm(promptMessage)

    return response

In [12]:
query('"作为顾客，我想将产品添加到购物车，以便稍后购买"。')

请分析用户故事,根据提示的答案写出用户故事的分数和评价，测试
1. 问题："作为顾客，我想按类别筛选产品，这样就能缩小搜索范围"。 || 答案：总分：79这个用户故事看起来结构很好，总分为79。它有效地传达了客户按类别筛选产品的需求，为评估和测试提供了足够的信息。此外，验收标准进一步提高了用户故事的清晰度和可测试性，确保开发团队在实现和验证所需功能方面有明确的指导。为用户指定的角色被明确地标识为客户的角色（10）。此外，角色的定义简洁明了，符合规定的5个单词的限制（3）。就用户故事组件而言，包含基本的“我想”条款可确保所需的动作得到明确表达（10）。预期结果，包括按类别过滤产品，得到了很好的阐述，并与客户的角色无缝一致（8）。此外，用户故事包含了关键的“所以”元素，为指定的操作提供了有价值的上下文（10）。向前看，缩小搜索范围的目标已明确传达（7）。用户故事成功地在客户的角色和期望的结果之间保持了清晰的相关性（8）。就实用性而言，用户故事可能很容易在开发工作方面得到评估（6）。此外，故事结构良好，易于测试，具有可测试的功能和潜在的测试场景（6）。验收标准：按类别过滤产品-阳性案例：测试案例：通过从可用选项中选择特定类别，验证客户能否成功过滤产品。小黄瓜测试：假设客户在产品列表页面上，当他们选择“电子”类别时，则显示的产品应仅限于“电子”类产品。按类别筛选产品-阴性案例（无结果）：测试案例：确保客户在尝试按没有匹配项目的类别筛选产品时收到适当的反馈。小黄瓜测试：假设客户在产品列表页面上，当他们选择“运动装备”类别时，他们应该会看到一条消息，表明所选类别中没有可用的产品。按类别筛选产品-多个类别：测试用例：验证客户是否可以通过同时选择多个类别来筛选产品。小黄瓜测试：假设客户在产品列表页面上，当他们同时选择“服装”和“鞋类”类别时，则显示的产品应包括两个选定类别的商品。按类别过滤产品-清除过滤器：测试用例：确认客户可以轻松清除所选类别过滤器以查看整个产品目录。小黄瓜测试：假设客户已按“家庭装饰”类别过滤产品，当他们单击“清除过滤器”按钮时，产品列表应恢复为显示所有可用类别的产品。
问题："作为顾客，我想将产品添加到购物车，以便稍后购买"。 || 答案：


The dtype of attention mask (torch.int64) is not bool
/Users/tianlimin/.cache/huggingface/modules/transformers_modules/chatglm-6b/modeling_chatglm.py:460: UserWarning: MPS: no support for int64 for min_max, downcasting to a smaller data type (int32/float32). Native support for int64 has been added in macOS 13.3. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:621.)
  cos, sin = self.rotary_emb(q1, seq_len=position_ids.max() + 1)


'Etat: 100这个用户故事看起来结构很好，总分为100。它有效地传达了客户添加产品到购物车的需求，为评估和测试提供了足够的信息。此外，验收标准进一步提高了用户故事的清晰度和可测试性，确保开发团队在实现和验证所需功能方面有明确的指导。为用户指定的角色被明确地标识为客户的角色(10)。添加产品到购物车是客户的角色(10)。预期结果，包括添加到购物车的产品，已被成功地传达(10)。购物车中的产品已被成功地添加到购物车(10)。向前看，目标已明确传达(10)。用户故事成功地在客户的角色和期望的结果之间保持了清晰的相关性(10)。就实用性而言，用户故事可能很容易在开发工作方面得到评估(6)。故事结构良好，易于测试，具有可测试的功能和潜在的测试场景(6)。验收标准：购物车-阳性案例：测试案例：通过从购物车列表中选择已添加到购物车的产品，验证客户是否成功保存购物车中的产品。小黄瓜测试：假设客户在产品列表页面上，当他们选择“食品”类别时，则显示的产品应仅限于“食品”类产品。小黄瓜测试：假设客户在购物车页面上，当他们选择“食品”和“饮料”类别时，则显示的产品应包括两个选定类别的商品。购物车-阴性案例(无结果)：测试案例：确保客户在尝试添加未匹配项目的类别时收到适当的反馈。小黄瓜测试：假设客户在购物车页面上，当他们选择“运动装备”类别时，他们应该会看到一条消息，表明所选类别中没有可用的产品。购物车-多个类别：测试用例：验证客户是否可以通过同时选择多个类别来添加产品到购物车。小黄瓜测试：假设客户在购物车页面上，当他们同时选择“服装”和 ⁇ ️“鞋类”类别时，则显示的产品应包括两个选定类别的商品。购物车-清除过滤器：测试用例：确认 ⁇ ️客户可以轻松清除所选类别过滤器以查看整个购物车列表。小黄瓜测试：假设客户已按“食品”类别过滤产品，当他们单击“清除过滤器”按钮时，购物车列表应恢复为显示所有可用类别的产品。\n\n总之，这个用户故事分数为100分，评价很高。'